In [3]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc
import os
import csv
import itertools
import tensorflow.contrib.slim as slim
%matplotlib inline

import taxi_env as te
import taxi_util as tu

# Environment

In [4]:
#define the input here
N_station=84
l1=[5 for i in range(N_station)]
OD_mat=[l1 for i in range(N_station)]
distance=OD_mat
travel_time=OD_mat
arrival_rate=[5 for i in range(N_station)]
taxi_input=6

env=te.taxi_simulator(arrival_rate,OD_mat,distance,travel_time,taxi_input)
env.reset()
print('System Successfully Initialized!')

System Successfully Initialized!


## Define the structure of the deep Q network

In [5]:
class Qnetwork():
    def __init__(self,N_station,h_size,rnn_cell,myScope):
        #The network recieves a frame from the game, flattened into an array.
        #It then resizes it and processes it through four convolutional layers.
        
        #input is a scalar which will later be reshaped
        self.scalarInput =  tf.placeholder(shape=[None,N_station*N_station*3],dtype=tf.float32)
        
        #input is a tensor, like a 3 chanel image
        self.imageIn = tf.reshape(self.scalarInput,shape=[-1,N_station,N_station,3]) 
        
        
        #create 4 convolution layers first
        
        self.conv1 = slim.convolution2d( \
            inputs=self.imageIn,num_outputs=32,\
            kernel_size=[8,8],stride=[4,4],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv1')
        self.conv2 = slim.convolution2d( \
            inputs=self.conv1,num_outputs=64,\
            kernel_size=[3,3],stride=[2,2],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv2')
        self.conv3 = slim.convolution2d( \
            inputs=self.conv2,num_outputs=64,\
            kernel_size=[3,3],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv3')
        self.conv4 = slim.convolution2d( \
            inputs=self.conv3,num_outputs=h_size,\
            kernel_size=[7,7],stride=[1,1],padding='VALID', \
            biases_initializer=None,scope=myScope+'_conv4')
        
        self.trainLength = tf.placeholder(dtype=tf.int32)
        #We take the output from the final convolutional layer and send it to a recurrent layer.
        #The input must be reshaped into [batch x trace x units] for rnn processing, 
        #and then returned to [batch x units] when sent through the upper levles.
        self.batch_size = tf.placeholder(dtype=tf.int32,shape=[])
        self.convFlat = tf.reshape(slim.flatten(self.conv4),[self.batch_size,self.trainLength,h_size])
        self.state_in = rnn_cell.zero_state(self.batch_size, tf.float32)
        self.rnn,self.rnn_state = tf.nn.dynamic_rnn(\
                inputs=self.convFlat,cell=rnn_cell,dtype=tf.float32,initial_state=self.state_in,scope=myScope+'_rnn')
        self.rnn = tf.reshape(self.rnn,shape=[-1,h_size])
        #The output from the recurrent player is then split into separate Value and Advantage streams
        self.streamA,self.streamV = tf.split(self.rnn,2,1)
        self.AW = tf.Variable(tf.random_normal([h_size//2,N_station]))
        self.VW = tf.Variable(tf.random_normal([h_size//2,1]))
        self.Advantage = tf.matmul(self.streamA,self.AW)
        self.Value = tf.matmul(self.streamV,self.VW)
        
        self.salience = tf.gradients(self.Advantage,self.imageIn)
        #Then combine them together to get our final Q-values.
        self.Qout = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))
        self.predict = tf.argmax(self.Qout,1)
        
        #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
        self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions_onehot = tf.one_hot(self.actions,N_station,dtype=tf.float32)
        
        self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
        self.td_error = tf.square(self.targetQ - self.Q)
        
        #In order to only propogate accurate gradients through the network, we will mask the first
        #half of the losses for each trace as per Lample & Chatlot 2016
        self.maskA = tf.zeros([self.batch_size,self.trainLength//2])
        self.maskB = tf.ones([self.batch_size,self.trainLength//2])
        self.mask = tf.concat([self.maskA,self.maskB],1)
        self.mask = tf.reshape(self.mask,[-1])
        self.loss = tf.reduce_mean(self.td_error * self.mask)
        
        self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
        self.updateModel = self.trainer.minimize(self.loss)

# Experience Replay

These classes allow us to store experies and sample then randomly to train the network.
Episode buffer stores experiences for each individal episode.
Experience buffer stores entire episodes of experience, and sample() allows us to get training batches needed from the network.

In [6]:
class experience_buffer():
    def __init__(self, buffer_size = 1000):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self,experience):
        if len(self.buffer) + 1 >= self.buffer_size:
            self.buffer[0:(1+len(self.buffer))-self.buffer_size] = []
        self.buffer.append(experience)
            
    def sample(self,batch_size,trace_length):
        sampled_episodes = random.sample(self.buffer,batch_size)
        sampledTraces = []
        for episode in sampled_episodes:
            point = np.random.randint(0,len(episode)+1-trace_length)
            sampledTraces.append(episode[point:point+trace_length])
        sampledTraces = np.array(sampledTraces)
        return np.reshape(sampledTraces,[batch_size*trace_length,4])

### Training the network

Network parameter here

In [7]:
#Setting the training parameters
batch_size = 4 #How many experience traces to use for each training step.
trace_length = 8 #How long each experience trace will be when training
update_freq = 5 #How often to perform a training step.
y = .99 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
anneling_steps = 10000 #How many steps of training to reduce startE to endE.
num_episodes = 1000 #How many episodes of game environment to train network with.
pre_train_steps = 10000 #How many steps of random actions before training begins.
load_model = False #Whether to load a saved model.
path = "./drqn" #The path to save our model to.
h_size = 128 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
max_epLength = 1000 #The max allowed length of our episode.
time_per_step = 1 #Length of each step used in gif creation
summaryLength = 100 #Number of epidoes to periodically save for analysis
tau = 0.001

In [ ]:
tf.reset_default_graph()
#We define the cells for the primary and target q-networks

#LSTM First
cell = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
cellT = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)

#DQN combined with LSTM
mainQN = Qnetwork(N_station,h_size,cell,'main')
targetQN = Qnetwork(N_station,h_size,cellT,'target')

init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=5)

trainables = tf.trainable_variables() #return the list of variables to train

targetOps = tu.updateTargetGraph(trainables,tau) #match the variables of target network with the origin one

myBuffer = experience_buffer()

#Set the rate of random action decrease. 
e = startE
stepDrop = (startE - endE)/anneling_steps

#create lists to contain total rewards and steps per episode
jList = []
rList = []
total_steps = 0

#Make a path for our model to be saved in.
if not os.path.exists(path):
    os.makedirs(path)

##Write the first line of the master log-file for the Control Center
with open('./Center/log.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(['Episode','Length','Reward','IMG','LOG','SAL'])    
  

with tf.Session() as sess:
    #this step loads the model from the model that has been saved
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    sess.run(init)
   
    #this example equals target network to the original network after every few episodes
    #we may want to modify this
    
    tu.updateTarget(targetOps,sess) #Set the target network to be equal to the primary network.
    for i in range(num_episodes):
        episodeBuffer = []
        
        #Reset environment and get first new observation
        env.reset() 
        
        #return the current state of the system
        sP,tempr=env.get_state() 
        print(sP.size)
        #process the state into a list
        s = tu.processState(sP,N_station)
        
        
        rAll = 0
        j = 0
        
        #Reset the recurrent layer's hidden state
        state = (np.zeros([1,h_size]),np.zeros([1,h_size])) 
        #The Q-Network
        while j < max_epLength: 
            j+=1
            print('Episode:',i,', ePLength:',j)
            
            #for all the stations, act greedily
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < e or total_steps < pre_train_steps:
                state1 = sess.run(mainQN.rnn_state,\
                    feed_dict={mainQN.scalarInput:[s],mainQN.trainLength:1,mainQN.state_in:state,mainQN.batch_size:1})
                a = np.random.randint(0,N_station)
            else:
                a, state1 = sess.run([mainQN.predict,mainQN.rnn_state],\
                    feed_dict={mainQN.scalarInput:[s],mainQN.trainLength:1,mainQN.state_in:state,mainQN.batch_size:1})
                a = a[0]
            
            #move to the next step based on action selected
            env.step([a])
            #get state and reward
            s1P,r=env.get_state()
            s1=tu.processState(s1P,N_station)
            
            total_steps += 1
            
            #episode buffer
            #we don't store the initial 200 steps of the simulation, as warm up periods
            if j>200:
                episodeBuffer.append(np.reshape(np.array([s,a,r,s1]),[1,4]))
            
            if total_steps > pre_train_steps:
            #start training here
                if e > endE:
                    e -= stepDrop

                if total_steps % (update_freq) == 0:
                    tu.updateTarget(targetOps,sess)  #update the target Q networks
                    
                    #Reset the recurrent layer's hidden state
                    state_train = (np.zeros([batch_size,h_size]),np.zeros([batch_size,h_size])) 
                    trainBatch = myBuffer.sample(batch_size,trace_length) #Get a random batch of experiences.
                    #Below we perform the Double-DQN update to the target Q-values
                    Q1 = sess.run(mainQN.predict,feed_dict={\
                        mainQN.scalarInput:np.vstack(trainBatch[:,3]),\
                        mainQN.trainLength:trace_length,mainQN.state_in:state_train,mainQN.batch_size:batch_size})
                    Q2 = sess.run(targetQN.Qout,feed_dict={\
                        targetQN.scalarInput:np.vstack(trainBatch[:,3]),\
                        targetQN.trainLength:trace_length,targetQN.state_in:state_train,targetQN.batch_size:batch_size})
                    end_multiplier = -(trainBatch[:,4] - 1)
                    doubleQ = Q2[range(batch_size*trace_length),Q1]
                    targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                    #Update the network with our target values.
                    sess.run(mainQN.updateModel, \
                        feed_dict={mainQN.scalarInput:np.vstack(trainBatch[:,0]),mainQN.targetQ:targetQ,\
                        mainQN.actions:trainBatch[:,1],mainQN.trainLength:trace_length,\
                        mainQN.state_in:state_train,mainQN.batch_size:batch_size})
            
            #update reward
            rAll += r
            
            #swap state
            s = s1
            sP = s1P
            state = state1
 

        #Add the episode to the experience buffer
        bufferArray = np.array(episodeBuffer)
        episodeBuffer = list(zip(bufferArray))
        myBuffer.add(episodeBuffer)
        jList.append(j)
        rList.append(rAll) #reward in this episode

        #Periodically save the model. 
        if i % 1000 == 0 and i != 0:
            #saver.save(sess,path+'/model-'+str(i)+'.cptk')
            print ("Saved Model")
        if len(rList) % summaryLength == 0 and len(rList) != 0:
            print (total_steps,np.mean(rList[-summaryLength:]), e)
#             saveToCenter(i,rList,jList,np.reshape(np.array(episodeBuffer),[len(episodeBuffer),5]),\
#                 summaryLength,h_size,sess,mainQN,time_per_step)
    #saver.save(sess,path+'/model-'+str(i)+'.cptk')

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Target Set Success
21168
Episode: 0 , ePLength: 1
Episode: 0 , ePLength: 2
Episode: 0 , ePLength: 3
Episode: 0 , ePLength: 4
Episode: 0 , ePLength: 5
Episode: 0 , ePLength: 6
Episode: 0 , ePLength: 7
Episode: 0 , ePLength: 8
Episode: 0 , ePLength: 9
Episode: 0 , ePLength: 10
Episode: 0 , ePLength: 11
Episode: 0 , ePLength: 12
Episode: 0 , ePLength: 13
Episode: 0 , ePLength: 14
Episode: 0 , ePLength: 15
Episode: 0 , ePLength: 16
Episode: 0 , ePLength: 17
Episode: 0 , ePLength: 18
Episode: 0 , ePLength: 19
Episode: 0 , ePLength: 20
Episode: 0 , ePLength: 21
Episode: 0 , ePLength: 22
Episode: 0 , ePLength: 23
Episode: 0 , ePLength: 24
Episode: 0 , ePLength: 25
Episode: 0 , ePLength: 26
Epi

Episode: 0 , ePLength: 288
Episode: 0 , ePLength: 289
Episode: 0 , ePLength: 290
Episode: 0 , ePLength: 291
Episode: 0 , ePLength: 292
Episode: 0 , ePLength: 293
Episode: 0 , ePLength: 294
Episode: 0 , ePLength: 295
Episode: 0 , ePLength: 296
Episode: 0 , ePLength: 297
Episode: 0 , ePLength: 298
Episode: 0 , ePLength: 299
Episode: 0 , ePLength: 300
Episode: 0 , ePLength: 301
Episode: 0 , ePLength: 302
Episode: 0 , ePLength: 303
Episode: 0 , ePLength: 304
Episode: 0 , ePLength: 305
Episode: 0 , ePLength: 306
Episode: 0 , ePLength: 307
Episode: 0 , ePLength: 308
Episode: 0 , ePLength: 309
Episode: 0 , ePLength: 310
Episode: 0 , ePLength: 311
Episode: 0 , ePLength: 312
Episode: 0 , ePLength: 313
Episode: 0 , ePLength: 314
Episode: 0 , ePLength: 315
Episode: 0 , ePLength: 316
Episode: 0 , ePLength: 317
Episode: 0 , ePLength: 318
Episode: 0 , ePLength: 319
Episode: 0 , ePLength: 320
Episode: 0 , ePLength: 321
Episode: 0 , ePLength: 322
Episode: 0 , ePLength: 323
Episode: 0 , ePLength: 324
E

Episode: 0 , ePLength: 597
Episode: 0 , ePLength: 598
Episode: 0 , ePLength: 599
Episode: 0 , ePLength: 600
Episode: 0 , ePLength: 601
Episode: 0 , ePLength: 602
Episode: 0 , ePLength: 603
Episode: 0 , ePLength: 604
Episode: 0 , ePLength: 605
Episode: 0 , ePLength: 606
Episode: 0 , ePLength: 607
Episode: 0 , ePLength: 608
Episode: 0 , ePLength: 609
Episode: 0 , ePLength: 610
Episode: 0 , ePLength: 611
Episode: 0 , ePLength: 612
Episode: 0 , ePLength: 613
Episode: 0 , ePLength: 614
Episode: 0 , ePLength: 615
Episode: 0 , ePLength: 616
Episode: 0 , ePLength: 617
Episode: 0 , ePLength: 618
Episode: 0 , ePLength: 619
Episode: 0 , ePLength: 620
Episode: 0 , ePLength: 621
Episode: 0 , ePLength: 622
Episode: 0 , ePLength: 623
Episode: 0 , ePLength: 624
Episode: 0 , ePLength: 625
Episode: 0 , ePLength: 626
Episode: 0 , ePLength: 627
Episode: 0 , ePLength: 628
Episode: 0 , ePLength: 629
Episode: 0 , ePLength: 630
Episode: 0 , ePLength: 631
Episode: 0 , ePLength: 632
Episode: 0 , ePLength: 633
E

Episode: 0 , ePLength: 908
Episode: 0 , ePLength: 909
Episode: 0 , ePLength: 910
Episode: 0 , ePLength: 911
Episode: 0 , ePLength: 912
Episode: 0 , ePLength: 913
Episode: 0 , ePLength: 914
Episode: 0 , ePLength: 915
Episode: 0 , ePLength: 916
Episode: 0 , ePLength: 917
Episode: 0 , ePLength: 918
Episode: 0 , ePLength: 919
Episode: 0 , ePLength: 920
Episode: 0 , ePLength: 921
Episode: 0 , ePLength: 922
Episode: 0 , ePLength: 923
Episode: 0 , ePLength: 924
Episode: 0 , ePLength: 925
Episode: 0 , ePLength: 926
Episode: 0 , ePLength: 927
Episode: 0 , ePLength: 928
Episode: 0 , ePLength: 929
Episode: 0 , ePLength: 930
Episode: 0 , ePLength: 931
Episode: 0 , ePLength: 932
Episode: 0 , ePLength: 933
Episode: 0 , ePLength: 934
Episode: 0 , ePLength: 935
Episode: 0 , ePLength: 936
Episode: 0 , ePLength: 937
Episode: 0 , ePLength: 938
Episode: 0 , ePLength: 939
Episode: 0 , ePLength: 940
Episode: 0 , ePLength: 941
Episode: 0 , ePLength: 942
Episode: 0 , ePLength: 943
Episode: 0 , ePLength: 944
E

Episode: 1 , ePLength: 219
Episode: 1 , ePLength: 220
Episode: 1 , ePLength: 221
Episode: 1 , ePLength: 222
Episode: 1 , ePLength: 223
Episode: 1 , ePLength: 224
Episode: 1 , ePLength: 225
Episode: 1 , ePLength: 226
Episode: 1 , ePLength: 227
Episode: 1 , ePLength: 228
Episode: 1 , ePLength: 229
Episode: 1 , ePLength: 230
Episode: 1 , ePLength: 231
Episode: 1 , ePLength: 232
Episode: 1 , ePLength: 233
Episode: 1 , ePLength: 234
Episode: 1 , ePLength: 235
Episode: 1 , ePLength: 236
Episode: 1 , ePLength: 237
Episode: 1 , ePLength: 238
Episode: 1 , ePLength: 239
Episode: 1 , ePLength: 240
Episode: 1 , ePLength: 241
Episode: 1 , ePLength: 242
Episode: 1 , ePLength: 243
Episode: 1 , ePLength: 244
Episode: 1 , ePLength: 245
Episode: 1 , ePLength: 246
Episode: 1 , ePLength: 247
Episode: 1 , ePLength: 248
Episode: 1 , ePLength: 249
Episode: 1 , ePLength: 250
Episode: 1 , ePLength: 251
Episode: 1 , ePLength: 252
Episode: 1 , ePLength: 253
Episode: 1 , ePLength: 254
Episode: 1 , ePLength: 255
E

Episode: 1 , ePLength: 529
Episode: 1 , ePLength: 530
Episode: 1 , ePLength: 531
Episode: 1 , ePLength: 532
Episode: 1 , ePLength: 533
Episode: 1 , ePLength: 534
Episode: 1 , ePLength: 535
Episode: 1 , ePLength: 536
Episode: 1 , ePLength: 537
Episode: 1 , ePLength: 538
Episode: 1 , ePLength: 539
Episode: 1 , ePLength: 540
Episode: 1 , ePLength: 541
Episode: 1 , ePLength: 542
Episode: 1 , ePLength: 543
Episode: 1 , ePLength: 544
Episode: 1 , ePLength: 545
Episode: 1 , ePLength: 546
Episode: 1 , ePLength: 547
Episode: 1 , ePLength: 548
Episode: 1 , ePLength: 549
Episode: 1 , ePLength: 550
Episode: 1 , ePLength: 551
Episode: 1 , ePLength: 552
Episode: 1 , ePLength: 553
Episode: 1 , ePLength: 554
Episode: 1 , ePLength: 555
Episode: 1 , ePLength: 556
Episode: 1 , ePLength: 557
Episode: 1 , ePLength: 558
Episode: 1 , ePLength: 559
Episode: 1 , ePLength: 560
Episode: 1 , ePLength: 561
Episode: 1 , ePLength: 562
Episode: 1 , ePLength: 563
Episode: 1 , ePLength: 564
Episode: 1 , ePLength: 565
E

Episode: 1 , ePLength: 836
Episode: 1 , ePLength: 837
Episode: 1 , ePLength: 838
Episode: 1 , ePLength: 839
Episode: 1 , ePLength: 840
Episode: 1 , ePLength: 841
Episode: 1 , ePLength: 842
Episode: 1 , ePLength: 843
Episode: 1 , ePLength: 844
Episode: 1 , ePLength: 845
Episode: 1 , ePLength: 846
Episode: 1 , ePLength: 847
Episode: 1 , ePLength: 848
Episode: 1 , ePLength: 849
Episode: 1 , ePLength: 850
Episode: 1 , ePLength: 851
Episode: 1 , ePLength: 852
Episode: 1 , ePLength: 853
Episode: 1 , ePLength: 854
Episode: 1 , ePLength: 855
Episode: 1 , ePLength: 856
Episode: 1 , ePLength: 857
Episode: 1 , ePLength: 858
Episode: 1 , ePLength: 859
Episode: 1 , ePLength: 860
Episode: 1 , ePLength: 861
Episode: 1 , ePLength: 862
Episode: 1 , ePLength: 863
Episode: 1 , ePLength: 864
Episode: 1 , ePLength: 865
Episode: 1 , ePLength: 866
Episode: 1 , ePLength: 867
Episode: 1 , ePLength: 868
Episode: 1 , ePLength: 869
Episode: 1 , ePLength: 870
Episode: 1 , ePLength: 871
Episode: 1 , ePLength: 872
E

Episode: 2 , ePLength: 149
Episode: 2 , ePLength: 150
Episode: 2 , ePLength: 151
Episode: 2 , ePLength: 152
Episode: 2 , ePLength: 153
Episode: 2 , ePLength: 154
Episode: 2 , ePLength: 155
Episode: 2 , ePLength: 156
Episode: 2 , ePLength: 157
Episode: 2 , ePLength: 158
Episode: 2 , ePLength: 159
Episode: 2 , ePLength: 160
Episode: 2 , ePLength: 161
Episode: 2 , ePLength: 162
Episode: 2 , ePLength: 163
Episode: 2 , ePLength: 164
Episode: 2 , ePLength: 165
Episode: 2 , ePLength: 166
Episode: 2 , ePLength: 167
Episode: 2 , ePLength: 168
Episode: 2 , ePLength: 169
Episode: 2 , ePLength: 170
Episode: 2 , ePLength: 171
Episode: 2 , ePLength: 172
Episode: 2 , ePLength: 173
Episode: 2 , ePLength: 174
Episode: 2 , ePLength: 175
Episode: 2 , ePLength: 176
Episode: 2 , ePLength: 177
Episode: 2 , ePLength: 178
Episode: 2 , ePLength: 179
Episode: 2 , ePLength: 180
Episode: 2 , ePLength: 181
Episode: 2 , ePLength: 182
Episode: 2 , ePLength: 183
Episode: 2 , ePLength: 184
Episode: 2 , ePLength: 185
E

Episode: 2 , ePLength: 453
Episode: 2 , ePLength: 454
Episode: 2 , ePLength: 455
Episode: 2 , ePLength: 456
Episode: 2 , ePLength: 457
Episode: 2 , ePLength: 458
Episode: 2 , ePLength: 459
Episode: 2 , ePLength: 460
Episode: 2 , ePLength: 461
Episode: 2 , ePLength: 462
Episode: 2 , ePLength: 463
Episode: 2 , ePLength: 464
Episode: 2 , ePLength: 465
Episode: 2 , ePLength: 466
Episode: 2 , ePLength: 467
Episode: 2 , ePLength: 468
Episode: 2 , ePLength: 469
Episode: 2 , ePLength: 470
Episode: 2 , ePLength: 471
Episode: 2 , ePLength: 472
Episode: 2 , ePLength: 473
Episode: 2 , ePLength: 474
Episode: 2 , ePLength: 475
Episode: 2 , ePLength: 476
Episode: 2 , ePLength: 477
Episode: 2 , ePLength: 478
Episode: 2 , ePLength: 479
Episode: 2 , ePLength: 480
Episode: 2 , ePLength: 481
Episode: 2 , ePLength: 482
Episode: 2 , ePLength: 483
Episode: 2 , ePLength: 484
Episode: 2 , ePLength: 485
Episode: 2 , ePLength: 486
Episode: 2 , ePLength: 487
Episode: 2 , ePLength: 488
Episode: 2 , ePLength: 489
E